In [1]:
import xarray as xr
import numpy as np
import numpy.matlib
import gc
from numba import jit
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy import stats
from sklearn.metrics import mean_squared_error
%matplotlib inline

讀取模式資料

In [2]:
%%time
file = ['lfpw6_1','lfpw7_1', 'cmcc3_1', 'dwd2_1', 'ukmo13_1','ec']
# file = ['lfpw6_1', 'ec', 'ukmo13_1']
system = {}
for f in file:
    ds = xr.open_dataarray(f'D:/share/{f}.nc')
    ds = ds.astype('float64')
    system.setdefault(f,ds)
    del ds

Wall time: 4.11 s


讀取觀測資料

In [3]:
obs = xr.open_dataarray('D:/share/obs_1.nc')
obs = obs.astype('float64')
obs = obs.sel(time=slice('1995-01','2014-12'))

將資料按照leadtime排序,並集合成一個dictionary

In [4]:
%%time
t2m = xr.concat((system['lfpw6_1'],system['lfpw7_1'],system['cmcc3_1'],system['dwd2_1'],system['ukmo13_1'],system['ec']),dim='sys', join='override')
step={}
for leadtime in range(6):
    data = t2m.isel(time=slice(12-leadtime,252-leadtime),step=leadtime)
    step.setdefault(f'{leadtime+1}',data)

Wall time: 2.08 s


將dictonary合併成一個xarray的資料

In [5]:
t2m = xr.concat((step['1'],step['2'],step['3'],step['4'],step['5'],step['6']),dim='step', join='override')

並重新排序xarray的dimensions

In [6]:
t2m = t2m.transpose('sys','time','step','latitude','longitude')

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: This DataArray contains multi-dimensional coordinates. In the future, these coordinates will be transposed as well unless you specify transpose_coords=False.
  """Entry point for launching an IPython kernel.


設定dimensions

In [7]:
latitude = t2m.latitude
longitude = t2m.longitude
step = t2m.step 
sys = list(range(len(file)))
time = obs.sel(time=slice('1996-01','2014-12')).time

將模式資料以及觀測資料整理成同樣的維度

In [8]:
t2m = t2m.data
obs = obs.data[np.newaxis,:,np.newaxis,:,:]

In [9]:
print(t2m.shape)
print(obs.shape)

(6, 240, 6, 181, 360)
(1, 240, 1, 181, 360)


先創立一個修正預報值的矩陣

In [10]:
calibrate_data = np.ones((1,228,6,181,360))

In [11]:
gc.collect()

120

---

calibrated by Linear Regression 

In [ ]:
%%time

model = LinearRegression()
for i in range(19):  #19
    t2m_traindata =  np.delete(t2m,list(range((i * 12),(12 * i) + 24)),axis=1)
    obs_traindata =  np.delete(obs,list(range((i * 12),(12 * i) + 24)),axis=1)
    
    t2m_testdata = t2m[:,(i * 12):(12 * i) + 24,:]
    obs_testdata = obs[:,(i * 12):(12 * i) + 24,:]
    
    # Ensemble mean
    t2m_traindata = t2m_traindata.mean(axis=0)
    obs_traindata = obs_traindata.mean(axis=0)
    t2m_testdata = t2m_testdata.mean(axis=0)
    obs_testdata = obs_testdata.mean(axis=0)
    
    #climate average
    t2m_train_mean = t2m_traindata.mean(axis=0)
    obs_train_mean = obs_traindata.mean(axis=0)
    
    t2m_traindata_anomaly = t2m_traindata - t2m_train_mean[np.newaxis,:]
    t2m_traindata_cycmean = t2m_traindata_anomaly.reshape(18,12,6,181,360).mean(axis=0)
    t2m_traindata_nomean = t2m_traindata_anomaly - np.tile(t2m_traindata_cycmean,(18,1,1,1))
    
    obs_traindata_anomaly = obs_traindata - obs_train_mean[:,np.newaxis,:]
    obs_traindata_cycmean = obs_traindata_anomaly.reshape(18,12,1,181,360).mean(axis=0)
    obs_traindata_nomean = obs_traindata_anomaly - np.tile(obs_traindata_cycmean,(18,1,1,1))
    
    t2m_test_anomaly = t2m_testdata - t2m_train_mean[np.newaxis,:]
    t2m_test_nomean = t2m_test_anomaly - np.tile(t2m_traindata_cycmean,(2,1,1,1))
    
    obs_test_anomaly = obs_testdata - obs_train_mean[:,np.newaxis,:]
    obs_test_nomean = obs_test_anomaly - np.tile(obs_traindata_cycmean,(2,1,1,1))
    for leadtime in range(6):  #6
        if  i==0 or i==19:
            predictor = obs_traindata_nomean[:-1 - leadtime,:]
            predictand = obs_traindata_nomean[leadtime + 1:,:]
            t2m_predictand = t2m_traindata_nomean[leadtime + 1:,:]

        else:
            before = obs_traindata_nomean[0:(i * 12) - leadtime - 1,:]     #被移除年份之前的觀測值(predictor)
            before2 = obs_traindata_nomean[0 + leadtime + 1:(i * 12),:]    #被移除年份之前的觀測值(predictand)
            t2m_before = t2m_traindata_nomean[0 + leadtime + 1:(i * 12),:] #被移除年份之前的模式值(predictand)
            after = obs_traindata_nomean[(i * 12):-leadtime - 1,:]         #被移除年份之後的觀測值(predictor)
            after2 = obs_traindata_nomean[(i * 12) + leadtime + 1:,:]      #被移除年份之後的觀測值(predictand)
            t2m_after = t2m_traindata_nomean[(i * 12) + leadtime + 1:,:]   #被移除年份之後的模式值(predictand)
            predictor = np.concatenate((before, after), axis=0)     #將觀測值的predictor合併
            predictand = np.concatenate((before2, after2), axis=0)  #將觀測值的predictand合併
            t2m_predictand = np.concatenate((t2m_before, t2m_after), axis=0)
        
        for lat in range(181):   #緯度181
            for lon in range(360):  #經度360
                train = np.concatenate((predictor[np.newaxis,:,0,lat,lon], t2m_predictand[np.newaxis,:,leadtime,lat,lon]),axis=0)
                model.fit(train.T, predictand[:,0,lat,lon].T)
                
                test = np.concatenate((obs_test_nomean[np.newaxis,11 - leadtime:-leadtime - 1,0,lat,lon], t2m_test_nomean[np.newaxis,12:,leadtime,lat,lon]),axis=0)
                obs_nomean_guess = model.predict(test.T)
                pred = obs_nomean_guess + obs_train_mean[:,lat,lon] + obs_traindata_cycmean[:,0,lat,lon]
                calibrate_data[:,i*12:(i*12)+12,leadtime,lat,lon] = pred.T

---

將修正結果存為Linear2的檔名

In [ ]:
np.save('Linear2',calibrate_data)    

將修正結果存為xarray的格式 , 並儲存為NC檔

In [ ]:
Linear2 = xr.DataArray(calibrate_data,coords=[[1],time,step,latitude,longitude],dims=['sys','time','step','latitude','longitude'])

In [ ]:
time

In [ ]:
Linear2.to_netcdf('Linear2.nc')

In [ ]:
del calibrate_data

In [ ]:
gc.collect()

---

In [12]:
ds = xr.open_dataarray('Linear2.nc')
ds

<xarray.DataArray (sys: 1, time: 228, step: 6, latitude: 181, longitude: 360)>
[89138880 values with dtype=float64]
Coordinates:
  * sys        (sys) int32 1
  * time       (time) datetime64[ns] 1996-01-01 1996-02-01 ... 2014-12-01
  * step       (step) timedelta64[ns] 31 days 59 days ... 151 days 181 days
  * latitude   (latitude) float64 90.0 89.0 88.0 87.0 ... -88.0 -89.0 -90.0
  * longitude  (longitude) float64 0.0 1.0 2.0 3.0 ... 356.0 357.0 358.0 359.0

In [13]:
ds = ds.mean(dim='sys')   #將ensemble取ensemble mean
ds

<xarray.DataArray (time: 228, step: 6, latitude: 181, longitude: 360)>
array([[[[-28.05891333, -28.05973771, -28.06044168, ..., -28.05577531,
          -28.05646553, -28.05805997],
         [-27.96451474, -27.96242891, -27.96072283, ..., -27.97316791,
          -27.97078899, -27.96773751],
         [-27.96042982, -27.94575361, -27.93945561, ..., -28.0098833 ,
          -27.99360115, -27.97852   ],
         ...,
         [-23.48398485, -23.5193185 , -23.55237756, ..., -23.38117234,
          -23.40893971, -23.4440936 ],
         [-24.15833803, -24.176813  , -24.19663848, ..., -24.12606018,
          -24.13358803, -24.14574207],
         [-25.69380166, -25.68463035, -25.67612015, ..., -25.71034044,
          -25.70852065, -25.70185052]],

        [[-27.00579118, -27.00635097, -27.00706393, ..., -27.00190493,
          -27.00177372, -27.00350928],
         [-26.88844636, -26.88436579, -26.88081042, ..., -26.90322546,
          -26.89885878, -26.89354005],
         [-26.62333314, -26.6112031 , -26.60639287, ..., -26.6784161 ,
          -26.66402137, -26.64577111],
         ...,
         [-23.75728865, -23.78467258, -23.80979959, ..., -23.66396461,
          -23.69098659, -23.71932376],
         [-24.48782719, -24.50108614, -24.51614266, ..., -24.45782501,
          -24.46658923, -24.47545452],
         [-26.05420613, -26.05192367, -26.05181322, ..., -26.04712191,
          -26.04693908, -26.04747451]],

        [[-26.54484907, -26.54559205, -26.54594432, ..., -26.54559052,
          -26.54542408, -26.54601038],
         [-26.37211618, -26.36586762, -26.36064252, ..., -26.39361216,
          -26.38673782, -26.37953642],
         [-25.98930788, -25.9727951 , -25.96429019, ..., -26.05505927,
          -26.03546767, -26.01412708],
         ...,
         [-23.73884711, -23.76471097, -23.78930652, ..., -23.64716767,
          -23.6710731 , -23.7004731 ],
         [-24.48582196, -24.50049122, -24.51549167, ..., -24.45919518,
          -24.46699232, -24.47553489],
         [-26.11103992, -26.10606795, -26.10373539, ..., -26.10517897,
          -26.10406961, -26.10497366]],

        [[-26.46864729, -26.46866336, -26.46888378, ..., -26.46833639,
          -26.46844017, -26.46864131],
         [-26.34756029, -26.34184253, -26.33614715, ..., -26.36824521,
          -26.36154852, -26.35462728],
         [-26.03277435, -26.01624026, -26.0066108 , ..., -26.09908619,
          -26.07876635, -26.05764829],
         ...,
         [-23.46281019, -23.48205726, -23.5018714 , ..., -23.37736932,
          -23.39959688, -23.42481728],
         [-24.21519052, -24.22424635, -24.23484957, ..., -24.194948  ,
          -24.20132009, -24.20771919],
         [-25.742769  , -25.74192081, -25.7433111 , ..., -25.74408035,
          -25.7425085 , -25.74165014]],

        [[-26.79135984, -26.7917955 , -26.79232117, ..., -26.79187636,
          -26.79182623, -26.79187307],
         [-26.70140121, -26.6971068 , -26.69352469, ..., -26.71586252,
          -26.71111582, -26.70610187],
         [-26.43896402, -26.4257075 , -26.42014253, ..., -26.493432  ,
          -26.47751414, -26.46044154],
         ...,
         [-23.58706489, -23.61694814, -23.64561763, ..., -23.48198711,
          -23.51412146, -23.54635073],
         [-24.27105368, -24.2842869 , -24.29973418, ..., -24.23330743,
          -24.24533565, -24.25788552],
         [-25.74303481, -25.74077263, -25.73940271, ..., -25.74588438,
          -25.74476675, -25.74302378]],

        [[-27.29323329, -27.29450829, -27.29524256, ..., -27.2920921 ,
          -27.29249082, -27.2936444 ],
         [-27.17643215, -27.17238659, -27.1689343 , ..., -27.1933511 ,
          -27.1878672 , -27.18200599],
         [-26.91007565, -26.89352735, -26.88545996, ..., -26.97372064,
          -26.95553476, -26.93463353],
         ...,
         [-24.05915736, -24.07954015, -24.0991341 , ..., -23.96226807,
          -23.99235262, -24.02101968],
         [-24.6906186 , -24.70250613, -24.71702049, ..., -24.65839757,
          -24.66862195, 

重新開啟一次觀測資料

In [14]:
obs = xr.open_dataarray('D:/share/obs_1.nc')
obs = obs.astype('float64')

將觀測資料選為跟修正資料相同的時間

In [15]:
obs=obs.sel(time=slice('1996-01','2014-12'))
obs_cycmean = obs.groupby('time.month').mean(dim='time')  #計算觀測資料的年循環(1~12月)
obs

<xarray.DataArray 't2m' (time: 228, latitude: 181, longitude: 360)>
array([[[-28.61230469, -28.61230469, -28.61230469, ..., -28.61230469,
         -28.61230469, -28.61230469],
        [-28.22363281, -28.21582031, -28.20800781, ..., -28.24121094,
         -28.23535156, -28.22949219],
        [-28.21777344, -28.19628906, -28.17871094, ..., -28.27246094,
         -28.25488281, -28.23730469],
        ...,
        [-24.98730469, -25.01660156, -25.04589844, ..., -24.86816406,
         -24.90136719, -24.93847656],
        [-25.67675781, -25.69238281, -25.70996094, ..., -25.63769531,
         -25.64941406, -25.66308594],
        [-27.05957031, -27.05957031, -27.05957031, ..., -27.05957031,
         -27.05957031, -27.05957031]],

       [[-29.63343811, -29.63343811, -29.63343811, ..., -29.63343811,
         -29.63343811, -29.63343811],
        [-29.27015686, -29.27015686, -29.26820374, ..., -29.27796936,
         -29.27601624, -29.27406311],
        [-28.80921936, -28.79554749, -28.78773499, ..., -28.86390686,
         -28.85023499, -28.83460999],
        ...,
        [-32.27992249, -32.33265686, -32.37953186, ..., -32.09242249,
         -32.14710999, -32.20570374],
        [-33.03382874, -33.05335999, -33.07289124, ..., -32.97523499,
         -32.99476624, -33.01429749],
        [-34.60218811, -34.60218811, -34.60218811, ..., -34.60218811,
         -34.60218811, -34.60218811]],

       [[-30.92474365, -30.92474365, -30.92474365, ..., -30.92474365,
         -30.92474365, -30.92474365],
        [-31.29388428, -31.2880249 , -31.28411865, ..., -31.30169678,
         -31.29974365, -31.2958374 ],
        [-31.1317749 , -31.11419678, -31.10247803, ..., -31.1630249 ,
         -31.15325928, -31.14349365],
        ...,
        [-48.82904053, -48.88372803, -48.92669678, ..., -48.64349365,
         -48.6942749 , -48.75091553],
        [-49.54779053, -49.5614624 , -49.57318115, ..., -49.52044678,
         -49.5302124 , -49.53997803],
        [-51.0692749 , -51.0692749 , -51.0692749 , ..., -51.0692749 ,
         -51.0692749 , -51.0692749 ]],

       ...,

       [[-12.45779419, -12.45779419, -12.45779419, ..., -12.45779419,
         -12.45779419, -12.45779419],
        [-12.00662231, -11.99490356, -11.98318481, ..., -12.04177856,
         -12.03005981, -12.01834106],
        [-11.29763794, -11.27224731, -11.24685669, ..., -11.39334106,
         -11.36013794, -11.32888794],
        ...,
        [-47.1882782 , -47.2195282 , -47.24687195, ..., -47.0554657 ,
         -47.09257507, -47.13163757],
        [-47.55351257, -47.56132507, -47.5710907 , ..., -47.5320282 ,
         -47.5398407 , -47.54570007],
        [-48.56913757, -48.56913757, -48.56913757, ..., -48.56913757,
         -48.56913757, -48.56913757]],

       [[-20.85244751, -20.85244751, -20.85244751, ..., -20.85244751,
         -20.85244751, -20.85244751],
        [-22.03213501, -22.01651001, -22.00088501, ..., -22.07901001,
         -22.06338501, -22.04776001],
        [-22.41885376, -22.38565063, -22.35244751, ..., -22.53994751,
         -22.50088501, -22.45986938],
        ...,
        [-32.83486938, -32.85244751, -32.86416626, ..., -32.75674438,
         -32.77627563, -32.79776001],
        [-33.44033813, -33.44815063, -33.45401001, ..., -33.42861938,
         -33.43252563, -33.43643188],
        [-34.51651001, -34.51651001, -34.51651001, ..., -34.51651001,
         -34.51651001, -34.51651001]],

       [[-20.77905273, -20.77905273, -20.77905273, ..., -20.77905273,
         -20.77905273, -20.77905273],
        [-21.24975586, -21.23413086, -21.21655273, ..., -21.29467773,
         -21.28100586, -21.26538086],
        [-20.79272461, -20.75561523, -20.72631836, ..., -20.91186523,
         -20.87280273, -20.83569336],
        ...,
        [-22.71459961, -22.73803711, -22.76147461, ..., -22.62475586,
         -22.65014648, -22.67749023],
        [-23.46655273, -23.48217773, -23.49780273, ..., -23.43530273,
         -23.44506836, -23.45678711],
        [-25.13842773, -25.13842773, -25.13842773, 

---

# 計算CORR 和 RMSE (未減年循環)

將修正值與觀測值轉為numpy陣列做計算

In [16]:
calibrate_data = ds.data
obs1 = obs.data

In [17]:
%%time
cor_coef = np.zeros((6,181,360))
rmse = np.zeros((6,181,360))
for leadtime in range(6):
    for lat in range(181):
        for lon in range(360):
            r, _= stats.pearsonr(obs1[:,lat,lon], calibrate_data[:,leadtime,lat,lon])     #計算觀測值與修正值的相關係數
            cor_coef[leadtime, lat, lon] = r
            
            r1 = np.sqrt(mean_squared_error(obs1[:,lat,lon], calibrate_data[:,leadtime,lat,lon]))   #計算觀測值與修正值的RMSE
            std = np.std(obs1[:,lat,lon])    #計算觀測值的標準差
            rmse[leadtime, lat, lon] = r1 / std    #將RMSE標準化

Wall time: 1min 39s


In [18]:
# r1 = np.sqrt(mean_squared_error(obs1[:,lat,lon], calibrate_data[:,leadtime,lat,lon]))

將相關係數與RMSE轉換為xarray格式資料

In [19]:
corr = xr.DataArray(cor_coef,coords=[step,latitude,longitude],dims=['step','latitude','longitude'])
rmse = xr.DataArray(rmse,coords=[step,latitude,longitude],dims=['step','latitude','longitude'])

---

In [20]:
import cartopy.crs as ccrs
import cartopy
import cartopy.feature as cfeat
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from ipywidgets import interact , fixed

## 相關係數

In [22]:
def myplot_corr(corr,i):
    corr = xr.DataArray(cor_coef,coords=[step,latitude,longitude],dims=['step','latitude','longitude'])
    cbar_kwargs = {'label': 'correlation coef.','shrink':0.8,'ticks': np.arange(0,1,.1 )}
    # for i in range(6):

            # 創建畫圖空間
    proj = ccrs.PlateCarree() #創建投影
    fig = plt.figure(figsize=(16,9)) #創建頁面
    ax = fig.subplots(1, 1, subplot_kw={'projection': proj}) #子圖

    ax.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                                      linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.xlabels_top = False #關閉頂端標籤
    gl.ylabels_right = False #關閉右側標籤
    gl.xformatter = LONGITUDE_FORMATTER #X軸設為經度格式
    gl.yformatter = LATITUDE_FORMATTER #Y軸設為緯度格式


    corr.isel(step=i).plot(cmap=plt.cm.jet,
                            cbar_kwargs=cbar_kwargs,
                            transform=ccrs.PlateCarree(),
                            vmin=0,
                            vmax=1
                            ) #也可以用Size=?設定圖片大小 robust=True改善離群值
#     plt.imshow

In [23]:
interact(myplot_corr,i=(0,5),corr=fixed(corr))

interactive(children=(IntSlider(value=2, description='i', max=5), Output()), _dom_classes=('widget-interact',)…

<function __main__.myplot_corr(corr, i)>

## RMSE

In [24]:
def myplot_rmse(rmse,i):
    cbar_kwargs = {'label': 'RMSE','shrink':0.8,'ticks': np.arange(0,2,.2 )}
#     for i in range(6):

        # 創建畫圖空間
    proj = ccrs.PlateCarree() #創建投影
    fig = plt.figure(figsize=(16,9)) #創建頁面
    ax = fig.subplots(1, 1, subplot_kw={'projection': proj}) #子圖

    ax.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                              linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.xlabels_top = False #關閉頂端標籤
    gl.ylabels_right = False #關閉右側標籤
    gl.xformatter = LONGITUDE_FORMATTER #X軸設為經度格式
    gl.yformatter = LATITUDE_FORMATTER #Y軸設為緯度格式


    rmse.isel(step=i).plot(cmap=plt.cm.jet,
                            cbar_kwargs=cbar_kwargs,
                            transform=ccrs.PlateCarree(),
                            vmin=0,
                            vmax=2
                            ) #也可以用Size=?設定圖片大小 robust=True改善離群值
#         plt.show()

In [25]:
interact(myplot_rmse,i=(0,5),rmse=fixed(rmse)) ;

interactive(children=(IntSlider(value=2, description='i', max=5), Output()), _dom_classes=('widget-interact',)…

---

# 計算CORR 和 RMSE (減年循環)

計算修正值之年循環(1~12月)

In [26]:
calibrate_data_cycmean = ds.groupby('time.month').mean(dim='time')

In [27]:
calibrate_data_cycmean.shape

(12, 6, 181, 360)

In [28]:
%%time
calibrate_data_no_cycmean = np.zeros((228,6,181,360))
obs_no_cycmean = np.zeros((228,181,360))

calibrate_data_no_cycmean = ds.data - np.tile(calibrate_data_cycmean,(1,19,1,1,1))
obs_no_cycmean = obs.data - np.tile(obs_cycmean.data,(19,1,1))

Wall time: 755 ms


In [29]:
print(calibrate_data_no_cycmean.shape)
print(obs_no_cycmean.shape)

(1, 228, 6, 181, 360)
(228, 181, 360)


In [30]:
%%time
cor_coef = np.zeros((6,181,360))
rmse = np.zeros((6,181,360))
for leadtime in range(6):
    for lat in range(181):
        for lon in range(360):
            r, p= stats.pearsonr(obs_no_cycmean[:,lat,lon], np.squeeze(calibrate_data_no_cycmean)[:,leadtime,lat,lon])
            cor_coef[leadtime, lat, lon] = r
            
            r1 = np.sqrt(mean_squared_error(obs_no_cycmean[:,lat,lon], np.squeeze(calibrate_data_no_cycmean)[:,leadtime,lat,lon]))
            std = np.std(obs_no_cycmean[:,lat,lon])
            rmse[leadtime, lat, lon] = r1 / std

Wall time: 1min 36s


In [31]:
corr = xr.DataArray(cor_coef,coords=[step,latitude,longitude],dims=['step','latitude','longitude'])
rmse = xr.DataArray(rmse,coords=[step,latitude,longitude],dims=['step','latitude','longitude'])

## 相關係數

In [32]:
interact(myplot_corr,i=(0,5),corr=fixed(corr))

interactive(children=(IntSlider(value=2, description='i', max=5), Output()), _dom_classes=('widget-interact',)…

<function __main__.myplot_corr(corr, i)>

## RMSE

In [33]:
interact(myplot_rmse, i=(0,5), rmse=fixed(rmse))

interactive(children=(IntSlider(value=2, description='i', max=5), Output()), _dom_classes=('widget-interact',)…

<function __main__.myplot_rmse(rmse, i)>